In [1]:
import spacy
from spacy.tokens import Token
from spacy import displacy
import pandas as pd
import pickle
import random
import tqdm

C:\Users\aubin\AppData\Roaming\Python\Python38\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\aubin\AppData\Roaming\Python\Python38\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\aubin\AppData\Roaming\Python\Python38\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
all_verbs = []

In [3]:
file = open("../stored_data/docs.obj",'rb')
docs = pickle.load(file)
file.close()

In [4]:
id = random.sample(list(docs.keys()), 1)
random_doc = docs[id[0]]
full_doc = ""
for sent in random_doc:
    full_doc += sent

In [5]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_lg")
entities = nlp.get_pipe('ner').labels

In [6]:
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [
    {"ENT_TYPE": {'IN': entities}}, 
    {"ENT_TYPE": {'NOT_IN': entities}, 'OP': '*'}, 
    {'POS': 'VERB'},
    {"ENT_TYPE": {'NOT_IN': entities}, 'OP': '*'}, 
    {"ENT_TYPE": {'IN': entities}}]
matcher.add("any_verb", [pattern])

In [15]:
doc = nlp("Fairchild Corp was acquired by Fujitsu. Fairchild Corp acquired Fujitsu.")

matches = matcher(doc)
e2e_relationship = []


for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

    is_passive = False

    for token in span:
        if token.pos_ == "VERB":
            verb = token.text
            #all_verbs.append(token)
        if "pass" in token.dep_:
            is_passive = True

    if is_passive:
        print('here')
        for token in span:
            if "subjpass" in token.dep_:
                obj = token
                obj_ent = token.label_
            elif "obj" in token.dep_:
                subj = token
                subj_ent = token.label_
    else:
        for token in span:
            if "obj" in token.dep_:
                obj = token
                obj_ent = token.label_
            elif "subj" in  token.dep_:
                subj = token
                subj_ent = token.label_


    e2e_relationship.append([obj, obj_ent, verb, subj, subj_ent])


print(e2e_relationship)

13200282023714757851 any_verb 1 6 Corp was acquired by Fujitsu
here


AttributeError: 'spacy.tokens.token.Token' object has no attribute 'label_'

In [28]:
all_verbs = []
for id, file in tqdm.tqdm(docs.items()):
    full_doc = ""
    for sent in file:
        full_doc += sent

    doc = nlp(full_doc)

    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span

    for token in span:
        if token.pos_ == "VERB":
            verb = token.text
            all_verbs.append(token)

    for token in span.ents:
        

100%|██████████| 8399/8399 [03:42<00:00, 37.74it/s]


In [ ]:
print(set(all_verbs))

In [35]:
similarity_matrix = [[] for x in range(len(set(all_verbs)))]

for idx, token1 in enumerate(set(all_verbs)):
    print(idx/len(set(all_verbs)), end="\r")
    for token2 in set(all_verbs):
        similarity_matrix[idx].append(token1.similarity(token2))


C:\Users\aubin\AppData\Local\Temp\ipykernel_16532\3095854996.py:6: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_matrix[idx].append(token1.similarity(token2))


KeyboardInterrupt: 

In [39]:
set(all_verbs)

{adapt,
 be,
 simplify,
 compare,
 import,
 account,
 employ,
 grow,
 see,
 say,
 lead,
 gather,
 agree,
 raise,
 maintain,
 say,
 have,
 expand,
 rise,
 compare,
 fail,
 indicate,
 rise,
 suffer,
 affect,
 decrease,
 complete,
 expect,
 watch,
 delay,
 say,
 say,
 make,
 refrain,
 think,
 be,
 act,
 sell,
 withstand,
 bring,
 expect,
 discuss,
 say,
 prompt,
 tap,
 collapse,
 store,
 place,
 say,
 dictate,
 hold,
 produce,
 combine,
 operate,
 comprise,
 say,
 include,
 note,
 dlrs,
 be,
 say,
 act,
 be,
 make,
 boost,
 occur,
 say,
 say,
 show,
 place,
 be,
 become,
 say,
 tell,
 fall,
 file,
 follow,
 force,
 create,
 say,
 be,
 sell,
 persist,
 say,
 have,
 say,
 receive,
 reduce,
 repeal,
 set,
 change,
 meet,
 slip,
 say,
 operate,
 be,
 fuel,
 approve,
 invest,
 include,
 have,
 pay,
 put,
 change,
 go,
 buy,
 like,
 lend,
 continue,
 represent,
 dlrs,
 draft,
 agree,
 sell,
 begin,
 begin,
 fit,
 fill,
 say,
 forecast,
 bothersomeonce,
 say,
 enjoin,
 be,
 be,
 ommitte,
 envisa

In [36]:
all_verbs_txt = [verb.text for verb in all_verbs]

In [37]:
filehandler = open("../stored_data/all_verbs.obj","wb")
pickle.dump(all_verbs_txt, filehandler)
filehandler.close()